In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-8un6fjek
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8un6fjek
  Resolved https://github.com/huggingface/transformers.git to commit b9da44bd3e45c2b2b77147a70c61c546faef9774
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7446857 sha256=c1491c4c4233f8025bda1d64bf3ea57dddd2485e12169cc91c8d19bbf91d4be0
  Stored in directory: /tmp/pip-ephem-wheel-cache-c8qrdw4s/wheels/cf/59/82/6492402e887a68975030bf8c06532260

In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("image_captioning_blip_notebook", framework="pytorch")

In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [5]:
data_location =  'Colab\ Notebooks/Flickr_8k_Dataset'

In [6]:
%cd drive/MyDrive/$data_location

/content/drive/MyDrive/Colab Notebooks/Flickr_8k_Dataset


In [7]:
%ls

101.jpg                    captions/      Cat03.jpg  iStock.jpg
3726025663_e7d35d23f6.jpg  captionss.txt  Images/


In [8]:
import os
import glob
from PIL import Image
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration

In [9]:
def read_photos_and_descriptions(file_path):
    photo_list = []
    description_list = []
    count = 0
    with open(file_path, "r") as file:
        for line in file:
          if count < 2000:
            photo, description = line.strip().split(",", 1)
            photo_list.append(photo)
            description_list.append(description)
          count += 1

    return photo_list, description_list

In [10]:
file_path = "captionss.txt"
photos, descriptions = read_photos_and_descriptions(file_path)

In [11]:
dataset = []
for i in range(len(photos)):
  lst = []
  image = Image.open('Images/'+photos[i])
  lst.append(image)
  lst.append(descriptions[i])
  dataset.append(lst)

KeyboardInterrupt: ignored

In [ ]:
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item[0], text=item[1], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)

In [ ]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in range(1):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

In [ ]:
# example = dataset[0]
# image = example[0]
# image
image = Image.open('3726025663_e7d35d23f6.jpg')
image

In [ ]:
inputs = processor(images=image, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values

generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)